In [ ]:
# import dependencies
import pandas as pd
import mysql.connector as mc
from datetime import datetime
import numpy as np
from bs4 import BeautifulSoup as Soup
import pdfkit
import win32com.client as wc
import xml.etree.ElementTree as ET
import sys
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import ssl
import smtplib
import os
import pysftp

# Get db credentials
from configTest import mysql_host, mysql_u, mysql_pw, vgc_host, vgc_u, vgc_pw, smtp_host, e_user, e_pw, port, sftp_h, sftp_u, sftp_p

In [ ]:
# Show all columns in DFs
pd.set_option("display.max_columns", None)

In [ ]:
# MySQL
def mysql_q (u, p, h, db, sql, cols, commit):
    # cols (0 = no, 1 = yes)
    # commit (select = 0, insert/update = 1)

    # connect to claim_qb_payments db
    cnx = mc.connect(user=u, password=p,
                    host=h,
                    database=db)
    cursor = cnx.cursor()

    # commit?
    if commit == 1:
        cursor.execute(sql)
        cnx.commit()
        sql_result = 0     
    else:
        cursor.execute(sql)
        sql_result = cursor.fetchall()

    # columns ?
    if cols == 1:
        columns=list([x[0] for x in cursor.description])
        # close connection
        cursor.close()
        cnx.close()
        # return query result [0] and columns [1]
        return sql_result
    else:
        # close connection
        cursor.close()
        cnx.close()
        # return query result
        return sql_result 

In [ ]:
# Email
def send_email(toEmail, subject, msg_html, attachPath='', *args):
    fromEmail = 'claims@visualgap.com'

    # address message
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = fromEmail
    msg['To'] = ','.join(toEmail)

    # create body
    body_html = MIMEText(msg_html, 'html')
    msg.attach(body_html) 

    for arg in args:
        fName = ''.join([attachPath, arg])
        filename = os.path.abspath(fName)

        with open(filename, 'rb') as fn:
            attachment = MIMEApplication(fn.read())
            attachment.add_header('Content-Disposition', 'attachment', filename=arg)
            msg.attach(attachment)

    context = ssl.create_default_context()
    try:
        server = smtplib.SMTP(smtp_host, port)
        # check connection
        server.ehlo()  
        # Secure the connection
        server.starttls(context=context)  
        # check connection
        server.ehlo()
        server.login(e_user, e_pw)
        # Send email
        server.sendmail(fromEmail, toEmail, msg.as_string())

    except Exception as e:
        # Print any error messages
        print(e)
    finally:
        server.quit()

In [ ]:
def clear_dir (path, ext):
    for x in os.listdir(path):
        if x.endswith(ext):
            os.remove(os.path.join(path, x))

In [ ]:
# Define directories
attachment_dir = 'S:/claims/letters/attachment/'
file_staging_dir = './letters/staging/'
# get current date
now = datetime.now()

In [ ]:
# Define file paths
attach_name = f'Claims_Paid_{now.strftime("%Y-%m-%d")}'
attach_file = f'{attachment_dir}{attach_name}'
html_file = f'{file_staging_dir}final_rpt.html'
err_html_file = f'{file_staging_dir}error.html'
csv_file = f'{file_staging_dir}final_rpt.csv'
err_csv_file = f'{file_staging_dir}error.csv'
template_file = './html/pymt_summary_template.html'
html_file2 = './letters/staging/Claims_Paid.html'

In [ ]:
# sql query to collect qb_txnid's
sql = '''
      SELECT rtbp_id, check_nbr, qb_txnid
      FROM ready_to_be_paid
      WHERE toVGC = 1
        AND qb_txnid <> '0';
      '''

In [ ]:
# save query results as DF
qb_df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql, 0, 0))

# add column names
qb_df_cols = ['rtbp_id','check_nbr', 'qb_txnid']
qb_df.columns = qb_df_cols

In [ ]:
# Connect to Quickbooks
try:
    sessionManager = wc.Dispatch("QBXMLRP2.RequestProcessor")    
    sessionManager.OpenConnection('', 'Claim Payments')
    ticket = sessionManager.BeginSession("", 2)
except Exception as e:
    print('''
    Make sure QuickBooks is running and you are logged into the Company File.
    ERROR: {}'''.format(e))
    sys.exit("Error with communicating with QuickBooks")

In [ ]:
# create qbxml to query qb for check numbers
try:
    for index, row in qb_df.iterrows():
        qbxmlQuery = '''
                    <?qbxml version="14.0"?>
                    <QBXML>
                        <QBXMLMsgsRq onError="stopOnError">
                            <CheckQueryRq>
                                <TxnID>{txnId}</TxnID> 
                            </CheckQueryRq>
                        </QBXMLMsgsRq>
                    </QBXML>
                    '''.format(txnId=row['qb_txnid'])

        # Send query and receive response
        responseString = sessionManager.ProcessRequest(ticket, qbxmlQuery)

        # output Check Number (RefNumber)
        QBXML = ET.fromstring(responseString)
        QBXMLMsgsRs = QBXML.find('QBXMLMsgsRs')
        checkResults = QBXMLMsgsRs.iter("CheckRet")
        chkNbr = '0'
        for checkResult in checkResults:
            chkNbr = checkResult.find('RefNumber').text

        # Add Check Number to ready_to_be_paid table
        qb_sql_file = '''UPDATE ready_to_be_paid
                    SET check_nbr = '{ChkNbr}'
                    WHERE rtbp_id = {rowID};'''.format(ChkNbr=chkNbr, rowID=row['rtbp_id'])
        
        # execute and commit sql
        mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', qb_sql_file, 0, 1)
except Exception as e:
    print('''
    Make sure to print checks and process ACH in Quickbooks prior to starting this process.
    ERROR: {}'''.format(e))
    sys.exit("Error with communicating with QuickBooks")

In [ ]:
# Disconnect from Quickbooks
sessionManager.EndSession(ticket)
sessionManager.CloseConnection()

In [ ]:
# sql query for GAP claims that are RTBP
sql = '''
      SELECT r.rtbp_id, r.claim_id, r.claim_nbr, r.carrier_id, r.lender_name, r.pymt_method, r.first, r.last, r.pymt_type_id, r.amount, r.payment_category_id, r.check_nbr, r.qb_txnid, r.pymt_date, r.toVGC, r.err_msg
      FROM ready_to_be_paid r
      INNER JOIN (SELECT batch_id
                  FROM ready_to_be_paid
                  WHERE toVGC = 1
                  GROUP BY batch_id) sq
      USING(batch_id);
      '''

In [ ]:
# save query results as DF
df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql, 0, 0))

In [ ]:
# add column names
df_cols = ['rtbp_id', 'claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'payment_category_id', 'check_nbr', 'qb_txnid', 'pymt_date', 'toVGC', 'err_msg']

df.columns = df_cols

In [ ]:
# Add carrier name
sql = '''
    SELECT carrier_id, description
    FROM carriers;
    '''
# save query results as DF
carrier_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql, 0, 0))

col_names = ['carrier_id', 'carrier']
carrier_df.columns = col_names

# Merge QB_ListID into df
df = df.merge(carrier_df, left_on='carrier_id', right_on='carrier_id').copy()


In [ ]:
# format df
# list of conditions
type_conds = [((df['payment_category_id'] == 1) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 1) & (df['pymt_type_id'] == 2)),
              ((df['payment_category_id'] == 2) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 2) & (df['pymt_type_id'] == 2)),
              ((df['payment_category_id'] == 3) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 3) & (df['pymt_type_id'] == 2))]
# list of name types
type_name = ['GAP', 'GAP Supp', 'GAP Plus', 'GAP Plus Supp', 'TotalRestart', 'TotalRestart Supp']

# add column and assigned values
df['Claim_Type'] = np.select(type_conds, type_name)

In [ ]:
# Create df to update VGC
# vgc_update_df = df[['claim_id', 'pymt_method', 'pymt_type_id', 'amount', 'payment_category_id', 'check_nbr', 'qb_txnid', 'pymt_date']].copy()

In [ ]:
# format amount
df['amount'] = df['amount'].map('${:,.2f}'.format)

In [ ]:
# Create df for Claim Payment Summary
final_rpt_df = df[['claim_nbr', 'carrier', 'lender_name', 'first', 'last', 'amount', 'pymt_method', 'check_nbr', 'pymt_date', 'Claim_Type']].loc[df['toVGC'] == 1].copy()

In [ ]:
# Format df
final_rpt_df.rename(columns = {'claim_nbr':'Claim Nbr', 'carrier':'Carrier', 'lender_name':'Lender', 'first':'First Name', 'last':'Last Name', 'amount':'Amount', 'pymt_method':'Method',
                               'check_nbr':'Check Nbr', 'pymt_date': 'Date', 'Claim_Type':'Claim Type'}, inplace=True)

In [ ]:
final_rpt_df.sort_values(by = ['Carrier', 'Last Name', 'First Name'], inplace=True)

In [ ]:
# create error_df
error_df = df[['claim_nbr', 'lender_name', 'first', 'last', 'err_msg']].loc[df['toVGC'] == 2].copy()

# Format df
error_df.rename(columns = {'claim_nbr':'Claim Nbr', 'lender_name':'Lender', 'first':'First Name', 'last':'Last Name', 'err_msg':'Error Message'}, inplace=True)

In [ ]:
# export is df as csv
final_rpt_df.to_csv(csv_file, index=False)
error_df.to_csv(err_csv_file, index=False)
# read in csv
csvFile = pd.read_csv(csv_file)
errCsvFile = pd.read_csv(err_csv_file)
# convert csv to html
csvFile.to_html(html_file, index=False)
errCsvFile.to_html(err_html_file, index=False)

In [ ]:
# Get html df
soup = Soup(open(html_file), "html.parser")
err_soup = Soup(open(err_html_file), "html.parser")
table = str(soup.select_one("table", {"class":"dataframe"}))
err_table = str(err_soup.select_one("table", {"class":"dataframe"}))

# Get template
soup2 = Soup(open(template_file), "html.parser")
# Find and insert payment table
df_div = soup2.find("div", {"id":"df"})
df_div.append(Soup(table, 'html.parser'))
# Find and insert error table
err_div = soup2.find("div", {"id":"error"})
err_div.append(Soup(err_table, 'html.parser'))

# write html file
with open(html_file2,'w') as file:
    file.write(str(soup2))

In [ ]:
# check if file exists
fName = ''.join([attach_file, '.pdf'])
fnNum = 0

while(os.path.isfile(fName) == True):
    fnNum += 1
    fName = ''.join([attach_file, '_', str(fnNum), '.pdf']) 


In [ ]:
# create PDF from html
pdf_options = {'orientation': 'landscape',
                'page-size': 'Letter',
                'margin-top': '0.25in',
                'margin-right': '0.25in',
                'margin-bottom': '0.25in',
                'margin-left': '0.25in',
                'encoding': "UTF-8",}
                
pdfkit.from_file(html_file2, fName, options=pdf_options)

In [ ]:
# Email Claim Summary Report
to = ['jared@visualgap.com']
sub = f'Claim Payment Summary {now.strftime("%Y-%m-%d")}'
msg_html = '''
            <html>
            <body>
                <p>Attached is the Claim Summary Report.<br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
           '''
if fnNum == 0:
    a_file = ''.join([attach_name, '.pdf'])
else:
    a_file = ''.join([attach_name, '_', str(fnNum), '.pdf'])

send_email(to, sub, msg_html, attachment_dir, a_file)

In [ ]:
# Create data file for SCC
scc_cols = ['Carrier', 'claim_nbr_end', 'blank1', 'policy', 'blank2', 'last', 'first', 'loss_type', 'loss_date', 'blank3', 'GAP_static', 'GAP_amount', 'blank4', 
            'blank5', 'blank6', 'lender', 'address', 'blank7', 'city', 'state', 'zip', 'status', 'blank8', 'GAP_static2', 'contract_id', 'vin', 'make',
            'model', 'year', 'claim_nbr_front', 'CHECK', 'pymt_code', 'blank9', 'blank10', 'PAID_static', 'status_date', 'blank11', 'status_date2', 'blank12',
            'FFS_static', 'chk_nbr']
scc_df = pd.DataFrame(columns = scc_cols) 

In [ ]:
# Get SCC claims
scc_rpt_df = df[['rtbp_id', 'claim_id', 'amount', 'check_nbr', 'Claim_Type']].loc[(df['toVGC'] == 1) & (df['carrier_id'] == 8)].copy()

In [ ]:
# format amount without $, commas, and decimals
scc_rpt_df.replace('[\$,\.]','', regex=True, inplace=True)

In [ ]:
# create scc_file df
scc_file_df = pd.DataFrame()

# Get SCC claim data
if len(scc_rpt_df) > 0:
    for index, row in scc_rpt_df.iterrows():
        # Check or '' & payment code
        if row['amount'] == '$0.00':
            check = ''
            p_code = '003'
        else:
            check = 'CHECK'
            p_code = '001'
        # Paid or Plus
        if 'Plus' in row['Claim_Type']:
            paid_or_plus = 'PLUS'
        else:
            paid_or_plus = 'PAID'
        # ACH in chk_nbr to 0 
        if 'ACH' in row['check_nbr']:
            chk_num = '0'
        else:
            chk_num = row['check_nbr']            
        # create query
        sql = '''
                SELECT 'SC' AS Carrier, 
                SUBSTRING(c.claim_nbr, CHAR_LENGTH(c.claim_nbr)-5, 5) AS claim_nbr_end,
                '' AS blank1,
                l.policy_nbr,
                '' AS blank2,
                SUBSTRING(b.last, 1, 30) AS lastn,
                SUBSTRING(b.first, 1, 30) AS firstn,
                CASE
                    WHEN c.loss_type_id=1 THEN 'CO'
                    WHEN c.loss_type_id=2 THEN 'TH'
                    WHEN c.loss_type_id=3 THEN 'WE'
                    ELSE 'OT'
                END AS loss_type,
                DATE_FORMAT(c.loss_date, "%Y%m%d") AS loss_date,
                '' AS blank3,
                'GAP' AS GAP_static,
                '{amt}' AS GAP_amount,
                '' AS blank4,
                '' AS blank5,
                '' AS blank6,
                SUBSTRING(l.alt_name, 1, 32) AS lender,
                SUBSTRING(l.address1, 1, 32) AS address,
                '' AS blank7,
                l.city,
                l.state,
                SUBSTRING(l.zip, 1, 5) AS zip,
                'C' AS status,
                '' AS blank8,
                'GAP' AS GAP_static2,
                c.contractId,
                v.vin,
                SUBSTRING(v.make, 1, 20) AS Make,
                SUBSTRING(v.model, 1, 20) AS Model,
                SUBSTRING(v.year, 2, 2) AS Year,
                SUBSTRING(c.claim_nbr, 1, 8) AS claim_nbr_front,
                '{chk}' AS Chk,
                '{pay_code}' AS pymt_code,
                '' AS blank9,
                '' AS blank10,
                '{paid_plus}' AS PAID_static,
                DATE_FORMAT(CURDATE(), "%Y%m%d") AS status_date,
                '' AS blank11,
                DATE_FORMAT(CURDATE(), "%Y%m%d") AS status_date2,
                '' AS blank12,
                'FFS' AS FFS_static,
                '{check_nbr}' AS Chk_nbr       
            FROM claims c
            INNER JOIN claim_lender l
                USING(claim_id)
            INNER JOIN claim_borrower b
                USING(claim_id)
            INNER JOIN claim_vehicle v
                USING(claim_id)
            WHERE c.claim_id = {claimId};
              '''.format(claimId=row['claim_id'], amt=row['amount'], chk=check, pay_code=p_code, paid_plus=paid_or_plus, check_nbr=chk_num)

        temp_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql, 0, 0))    
        scc_file_df = scc_file_df.append(temp_df)

In [ ]:
# fill each field has an exact starting position
#A
scc_file_df[0] = scc_file_df[0].str.pad(2, side='right', fillchar=' ')
#B
scc_file_df[1] = scc_file_df[1].str.pad(8, side='right', fillchar=' ')
#C
scc_file_df[2] = scc_file_df[2].str.pad(12, side='right', fillchar=' ')
#D
scc_file_df[3] = scc_file_df[3].str.pad(15, side='right', fillchar=' ')
#E
scc_file_df[4] = scc_file_df[4].str.pad(32, side='right', fillchar=' ')
#F
scc_file_df[5] = scc_file_df[5].str.pad(30, side='right', fillchar=' ')
#G
scc_file_df[6] = scc_file_df[6].str.pad(30, side='right', fillchar=' ')
#H
scc_file_df[7] = scc_file_df[7].str.pad(2, side='right', fillchar=' ')
#I
scc_file_df[8] = scc_file_df[8].str.pad(8, side='right', fillchar=' ')
#J
scc_file_df[9] = scc_file_df[9].str.pad(9, side='right', fillchar=' ')
#K
scc_file_df[10] = scc_file_df[10].str.pad(3, side='right', fillchar=' ')
#L
scc_file_df[11] = scc_file_df[11].str.pad(9, side='left', fillchar='0')
#M
scc_file_df[12] = scc_file_df[12].str.pad(9, side='right', fillchar=' ')
#N
scc_file_df[13] = scc_file_df[13].str.pad(9, side='right', fillchar=' ')
#O
scc_file_df[14] = scc_file_df[14].str.pad(10, side='right', fillchar=' ')
#P
scc_file_df[15] = scc_file_df[15].str.pad(32, side='right', fillchar=' ')
#Q
scc_file_df[16] = scc_file_df[16].str.pad(32, side='right', fillchar=' ')
#R
scc_file_df[17] = scc_file_df[17].str.pad(32, side='right', fillchar=' ')
#S
scc_file_df[18] = scc_file_df[18].str.pad(30, side='right', fillchar=' ')
#T
scc_file_df[19] = scc_file_df[19].str.pad(2, side='right', fillchar=' ')
#U
scc_file_df[20] = scc_file_df[20].str.pad(9, side='right', fillchar=' ')
#V
scc_file_df[21] = scc_file_df[21].str.pad(1, side='right', fillchar=' ')
#W
scc_file_df[22] = scc_file_df[22].str.pad(1, side='right', fillchar=' ')
#X
scc_file_df[23] = scc_file_df[23].str.pad(3, side='right', fillchar=' ')
#Y
scc_file_df[24] = scc_file_df[24].str.pad(20, side='right', fillchar=' ')
#Z
scc_file_df[25] = scc_file_df[25].str.pad(18, side='right', fillchar=' ')
#AA
scc_file_df[26] = scc_file_df[26].str.pad(20, side='right', fillchar=' ')
#AB
scc_file_df[27] = scc_file_df[27].str.pad(20, side='right', fillchar=' ')
#AC
scc_file_df[28] = scc_file_df[28].str.pad(2, side='right', fillchar=' ')
#AD
scc_file_df[29] = scc_file_df[29].str.pad(8, side='right', fillchar=' ')
#AE
scc_file_df[30] = scc_file_df[30].str.pad(20, side='right', fillchar=' ')
#AF
scc_file_df[31] = scc_file_df[31].str.pad(3, side='right', fillchar=' ')
#AG
scc_file_df[32] = scc_file_df[32].str.pad(20, side='right', fillchar=' ')
#AH
scc_file_df[33] = scc_file_df[33].str.pad(8, side='right', fillchar=' ')
#AI
scc_file_df[34] = scc_file_df[34].str.pad(20, side='right', fillchar=' ')
#AJ
scc_file_df[35] = scc_file_df[35].str.pad(8, side='right', fillchar=' ')
#AK
scc_file_df[36] = scc_file_df[36].str.pad(30, side='right', fillchar=' ')
#AL
scc_file_df[37] = scc_file_df[37].str.pad(8, side='right', fillchar=' ')
#AM
scc_file_df[38] = scc_file_df[38].str.pad(10, side='right', fillchar=' ')
#AN
scc_file_df[39] = scc_file_df[39].str.pad(3, side='right', fillchar=' ')
#AO
scc_file_df[40] = scc_file_df[40].str.pad(8, side='left', fillchar='0')


In [ ]:
# write text file
with open(attachment_dir + 'FrostGAP.txt', 'a') as f:
    for index, row in scc_file_df.iterrows():
        col_index = 0
        while col_index != len(row):
            f.write(row[col_index])
            col_index += 1
        f.write('\n')
f.close()

In [ ]:
# Disable host key checking
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

# Send file to SCC via SFTP
with pysftp.Connection(sftp_h, username=sftp_u, password=sftp_p, cnopts=cnopts) as sftp:
    with sftp.cd('dropoff'):
        sftp.put(attachment_dir + 'FrostGAP.txt')

In [ ]:
# Send Email notification to SCC and Claims
to = ['jared@visualgap.com']
sub = f'Frost claim file'
msg_html = '''
            <html>
            <body>
                <p>Hello,<br>
                <br>
                We have submitted a new claim file today.  If you have any questions or concerns please contact us. <br>
                <br>
                Thank you, <br>
                Claims Department <br>
                <br>
                <b>Frost Financial Services, Inc. | VisualGAP <br>
                Claims Department <br>
                Phone: 888-753-7678 Option 3</b>
                </p>
            </body>
            </html>
           '''
send_email(to, sub, msg_html)

In [ ]:
# update toVGC to 3
if len(df) > 0:
    for index, row in df.iterrows():
        err_sql = '''
                UPDATE ready_to_be_paid
                SET toVGC = 3
                WHERE rtbp_id = {rtbp_id};
                '''.format(rtbp_id=row['rtbp_id'])
        # run update query        
        mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', err_sql, 0, 1)

In [ ]:
# Remove files from staging directory
file_ext = [".csv", ".html"]
file_staging_dir = './letters/staging/'

for ext in file_ext:
    clear_dir(file_staging_dir, ext)
